# 数据下载

In [7]:
!mkdir -p ./data/hp/summary
!aws s3 cp s3://datalab2021/hupo_nlp/data/news_summary_total.csv ./data/hp/summary

download: s3://datalab2021/hupo_nlp/data/news_summary_total.csv to data/hp/summary/news_summary_total.csv


# 数据预处理

In [8]:
from sklearn.model_selection import train_test_split
import random
import torch
import re
import os
import string
import datasets
from datasets import load_dataset, load_metric,DatasetDict
from sklearn.model_selection import train_test_split
import pandas as pd

In [9]:
class Settings:


    TRAIN_DATA = "./data/hp/summary/news_summary_total.csv"

    Columns = ['headlines', 'text']

    encoding = 'latin-1'
#     columns_dict = {"headlines": "target_text", "text": "source_text"}
    columns_dict = {"headlines": "headlines", "text": "text"}
    df_column_list = ['text', 'headlines']
    SUMMARIZE_KEY = ""
    SOURCE_TEXT_KEY = 'text'
    TEST_SIZE = 0.2
    BATCH_SIZE = 16
    source_max_token_len = 128
    target_max_token_len = 50
    train_df_len = 82332
    test_df_len = 20583
class Preprocess:
    def __init__(self):
        self.settings = Settings

    def clean_text(self, text):
        text = text.lower()
        text = re.sub('\[.*?\]', '', text)
        text = re.sub('https?://\S+|www\.\S+', '', text)
        text = re.sub('<.*?>+', '', text)
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
        text = re.sub('\n', '', text)
        text = re.sub('\w*\d\w*', '', text)
        return text

    def preprocess_data(self, data_path):
        df = pd.read_csv(data_path, encoding=self.settings.encoding, usecols=self.settings.Columns)
        # simpleT5 expects dataframe to have 2 columns: "source_text" and "target_text"
        df = df.rename(columns=self.settings.columns_dict)
        df = df[self.settings.df_column_list]
        # T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
        df[self.settings.SOURCE_TEXT_KEY] = df[self.settings.SOURCE_TEXT_KEY]

        return df

In [10]:
settings=Settings
preprocess=Preprocess()
df = preprocess.preprocess_data(settings.TRAIN_DATA)
df.head

<bound method NDFrame.head of                                                      text  \
0       Saurav Kant, an alumnus of upGrad and IIIT-B's...   
1       Kunal Shah's credit card bill payment platform...   
2       New Zealand defeated India by 8 wickets in the...   
3       With Aegon Life iTerm Insurance plan, customer...   
4       Speaking about the sexual harassment allegatio...   
...                                                   ...   
102910  Fruit juice concentrate maker Rasna is eyeing ...   
102911  Former Indian cricketer Sachin Tendulkar atten...   
102912  Aamir Khan, while talking about reality shows ...   
102913  The Maharashtra government has initiated an in...   
102914  At least 400 languages or more than half langu...   

                                                headlines  
0       upGrad learner switches to career in ML & Al w...  
1       Delhi techie wins free food from Swiggy for on...  
2       New Zealand end Rohit Sharma-led India's 12-ma...

# 训练集-测试集划分

In [14]:
df.to_csv('./data/hp/summary/news_summary_cleaned.csv',index=False)
df2=pd.read_csv('./data/hp/summary/news_summary_cleaned.csv')
order=['text','headlines']
df3=df2[order]
train_df, test_df = train_test_split(df3, test_size=0.2,random_state=100)
train_df.shape,test_df.shape

((82332, 2), (20583, 2))

In [15]:
train_df.to_csv('./data/hp/summary/news_summary_cleaned_train.csv',index=False)
test_df.to_csv('./data/hp/summary/news_summary_cleaned_test.csv',index=False)
train_df[:1000].to_csv('./data/hp/summary/news_summary_cleaned_small_train.csv',index=False)
test_df[:100].to_csv('./data/hp/summary/news_summary_cleaned_small_test.csv',index=False)